MEC加速尝试 Python实现

In [4]:
import numpy as np
from scipy import sparse

In = np.array([[0,0,0,0,0,0,0],[0,2,2,1,1,2,0],[0,2,0,1,1,0,0,],[0,2,0,1,2,0,0],[0,1,1,1,1,1,0],[0,0,0,1,0,2,0],[0,0,0,0,0,0,0]])
fitter = np.array([[1,0,0],[1,1,1],[1,0,-1]])
print(In)
print(fitter)

[[0 0 0 0 0 0 0]
 [0 2 2 1 1 2 0]
 [0 2 0 1 1 0 0]
 [0 2 0 1 2 0 0]
 [0 1 1 1 1 1 0]
 [0 0 0 1 0 2 0]
 [0 0 0 0 0 0 0]]
[[ 1  0  0]
 [ 1  1  1]
 [ 1  0 -1]]


In [2]:
def mec_conv2D(In,fitter):
    #数据维度转换
    I = In.shape
    K = fitter.shape
    s = 1
    Ow = I[0]*K[0]
    Oh = int(((I[1]-K[1])/s) +1)
    O = np.zeros((int(Oh),int(Ow)))
    w = 0
    h=0
    i=0
    for oh in range(Oh):
        for ow in range(Ow):
            if w<K[1]+i:
                O[oh][ow] = In[h][w]
                w+=1
                if w == K[1]+i:
                    w=i
                    h+=1
                if h == I[0]:
                        h=0
        i+=1
        w+=1
    #print(O)
    #print(O.shape)

    #卷积核的维度转换
    Kk=np.zeros((K[0]*K[1],1))
    i=0
    for k1 in range(K[0]):
        for k2 in range(K[1]):
            Kk[i][0] = fitter[k1][k2]
            i+=1
    #print(Kk)
    #print(Kk.shape)
    
    #维度转换后进行数据的计算
    #MEC转换成为数据之后，进行计算之前确定0值，虽然增加了矩阵运算的数目但是减少乘法运算的次数
    o_w = int(((I[1]-K[1])/s) +1)
    o_h = int(((Ow-K[0]*K[1])/K[0])+1)
    k_shape = Kk.shape
    out = np.zeros((o_h,o_w))
    x,l = 0,0
    n = 0
    for i in range(o_h):
        for j in range(o_w):
            for k in range(k_shape[0]):
                #if Kk[k][0] !=0 and O[x][l] !=0:
                out[i][j] += Kk[k][0] * O[x][l]  
                l+=1
            l=n
            x+=1
        x=0
        n+=3
        l=n  
    #print(out)

In [3]:
mec_conv2D(In,fitter)#终于复现了论文代码

In [9]:
def mec_conv2D_v2(In,fitter):
    #数据维度转换
    I = In.shape
    K = fitter.shape
    s = 1
    Ow = I[0]*K[0]
    Oh = int(((I[1]-K[1])/s) +1)
    O = np.zeros((int(Oh),int(Ow)))
    w = 0
    h=0
    i=0
    for oh in range(Oh):
        for ow in range(Ow):
            if w<K[1]+i:
                O[oh][ow] = In[h][w]
                w+=1
                if w == K[1]+i:
                    w=i
                    h+=1
                if h == I[0]:
                        h=0
        i+=1
        w+=1
    print(O)
    #print(O.shape)

    #卷积核的维度转换
    Kk=np.zeros((K[0]*K[1],1))
    i=0
    for k1 in range(K[0]):
        for k2 in range(K[1]):
            Kk[i][0] = fitter[k1][k2]
            i+=1
    print(Kk)
    #print(Kk.shape)
    
    #维度转换后进行数据的计算
    #MEC转换成为数据之后，进行计算之前确定0值，虽然增加了矩阵运算的数目但是减少乘法运算的次数
    o_w = int(((I[1]-K[1])/s) +1)
    o_h = int(((Ow-K[0]*K[1])/K[0])+1)
    k_shape = Kk.shape
    out = np.zeros((o_h,o_w))
    x,l = 0,0
    n = 0
    for i in range(o_h):
        for j in range(o_w):
            for k in range(k_shape[0]):
                #if Kk[k][0] !=0 and O[x][l] !=0:
                out[i][j] += Kk[k][0] * O[x][l]  
                l+=1
            l=n
            x+=1
        x=0
        n+=3
        l=n  
    #print(out)
mec_conv2D_v2(In,fitter)

[[0. 0. 0. 0. 2. 2. 0. 2. 0. 0. 2. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 2. 1. 2. 0. 1. 2. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 1. 1. 0. 1. 1. 0. 1. 2. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 2. 1. 1. 0. 1. 2. 0. 1. 1. 1. 1. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 2. 0. 1. 0. 0. 2. 0. 0. 1. 1. 0. 0. 2. 0. 0. 0. 0.]]
[[ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [-1.]]


In [4]:
#筛选数据，筛选稀疏数据
def cool_matrix(In):
    in_shape = In.shape
    row=[]
    col=[]
    data=[]
    for i in range(in_shape[0]):
        for j in range(in_shape[1]):
            if In[i][j] != 0:
                row.append(i)
                col.append(j)
                data.append(In[i][j])
    return row,col,data

In [5]:
row,col,data=cool_matrix(In)
c = sparse.coo_matrix((data,(row,col)),shape=(7,7))

In [6]:
print(c.toarray())

[[0 0 0 0 0 0 0]
 [0 2 2 1 1 2 0]
 [0 2 0 1 1 0 0]
 [0 2 0 1 2 0 0]
 [0 1 1 1 1 1 0]
 [0 0 0 1 0 2 0]
 [0 0 0 0 0 0 0]]


In [7]:

#print(Kk.shape)

In [8]:
def row_num(row,col,data,Kk):
    out=np.zeros(5)
    l=0
    i=0
    for j in col:
        i=row[l]
        out[i] += data[l] *  Kk[j][0]
        l+=1    
    return out

In [7]:
def new_mec_conv2D(In,fitter):
    I = In.shape
    K = fitter.shape
    s = 1
    Ow = I[0]*K[0]
    Oh = int(((I[1]-K[1])/s) +1)
    O = np.zeros((int(Oh),int(Ow)))
    w = 0
    h=0
    i=0
    for oh in range(Oh):
        for ow in range(Ow):
            if w<K[1]+i:
                O[oh][ow] = In[h][w]
                w+=1
                if w == K[1]+i:
                    w=i
                    h+=1
                if h == I[0]:
                        h=0
        i+=1
        w+=1
    print(O)
    #print(O.shape)

    #卷积核的维度转换
    Kk=np.zeros((K[0]*K[1],1))
    i=0
    for k1 in range(K[0]):
        for k2 in range(K[1]):
            Kk[i][0] = fitter[k1][k2]
            i+=1
    temp = np.zeros((5,5,9))
    q,l,n=0,0,0
    for x in range(5):
        for i in range(5):
            for j in range(9):
                temp[x][i][j] = O[q][l] 
                l+=1
            l=n
            q+=1
        q=0
        n+=3
        l=n
    #print(temp[0])
    n = temp.shape
    out=np.zeros((5,5))
    for i in range(n[0]):
        row,col,data=cool_matrix(temp[i])
        #print(row)
        out[i] = row_num(row,col,data,Kk)
   # print(out)

In [8]:
new_mec_conv2D(In,fitter)   #CPU实现了自己的想法

[[0. 0. 0. 0. 2. 2. 0. 2. 0. 0. 2. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 2. 1. 2. 0. 1. 2. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 1. 1. 0. 1. 1. 0. 1. 2. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 2. 1. 1. 0. 1. 2. 0. 1. 1. 1. 1. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 2. 0. 1. 0. 0. 2. 0. 0. 1. 1. 0. 0. 2. 0. 0. 0. 0.]]


NameError: name 'cool_matrix' is not defined

In [14]:
import time
start2 = time.clock()
new_mec_conv2D(In,fitter)
end2 = time.clock()
t2 = end2 - start2

start1 = time.clock()
mec_conv2D(In,fitter)
end1 = time.clock()
t1 = end1 - start1

print(t1)
print(t2)

0.0007690000000000197
0.001454999999999984


In [15]:
#直接计算在CPU上面耗费的时间相对于矩阵的转换计算的时间少